## Sorting: Applications

----

In [1]:
## Define some function useful for testing
import random
import heapq

## Generate an array of n random integers up to b
def get_random_array(n, b = 50):
    return [random.randint(0, b) for _ in range(n)]

---

### Exercise: K-largest elements of an array

We want to compute the K-largest elements of a array A. **Report them sorted**.

There are three possible algorithms to solve this problem:


#### Algorithm 1: Sorting
The easiest way to solve this is by sorting the array in decreasing order and reporting the first K elements. 

This algorithm costs $\Theta(n\log n)$ time. 

Implement this algorithm in a function ```k_largest_sort(A, K)```and test its correctness.

#### Algorithm 2: QuickSelect
Implement the QuickSelect algorithm and use it to find the K-largest element E in the array A. Then, scan A again 
to collect the K elements larger than or equal to E. Finally, sort the collected elements.

This algorithm costs $\Theta(n + K\log K)$ time (in expectation). 

Implement this algorithm in a function ```k_largest_quickselect(A, K)```and test its correctness.


#### Algorithm 3: Heap
You have to implement the following faster algorithm as a function ```k_largest_heap(A,K)```.
- Scan the array from left to right and keep a min-heap. The min-heap will contain at most K elements.
- Insert the current element into the heap, if the heap has less than K elements or the current element is larger than the minimum in the heap. If the heap has more than K elements, remove the minimum. 
- Sort the collected elements.

This algorithm runs in $\Theta(n\log K)$ time.

Implement this algorithm in a function ```k_largest_heap(A, K)```and test its correctness.


**Compare the three solutions by varying the size of the array and the value K. Which one is the fastest?**

In [2]:
def k_largest_sort(a,k):
    return sorted(a)[-k:]

############################################################

def QuickSelect(a, k):
    if len(a) == 1:
        return a[0]

    pivot = a[0]
    left = [num for num in a[1:] if num <= pivot]
    right = [num for num in a[1:] if num > pivot]

    if k <= len(right):
        return QuickSelect(right, k)
    elif k == len(right) + 1:
        return pivot
    else:
        return QuickSelect(left, k - len(right) - 1)
    
def k_largest_quickselect(a,k):
    k_largest = []
    elem = QuickSelect(a,k)
    for item in a:
        if item >= elem:
            k_largest.append(item)
    return k_largest

############################################################

def k_largest_heap(a,k):
    min_heap = [] 
    for elem in a: 
        if (len(min_heap) <= k) or (elem > min_heap[0]): 
            heapq.heappush(min_heap, elem) # insert the current element into the heap
        if len(min_heap) > k: 
            heapq.heappop(min_heap) # remove the minimum
    return sorted(min_heap, reverse = True)

The **QuickSelect** function uses a modified version of the 'traditional' Quickselect algorithm to find the 'kth' largest element in the input list 'a'. It recursively partitions the list around a pivot element and then narrows down the search to the left or right partition based on the value of 'k'. If 'k' is less than or equal to the length of the right partition, the search continues in the right partition, otherwise the search continues in the left partition with an updated value of 'k'. The function returns the 'kth' largest element in the list.  
This modified version was obtained by simply modifying the original version and reversion the check of 'kth' element with the left and right part.

The **k_largest_quickselect** function uses MaxElement to find the 'kth' largest element in the input list 'a' and then returns all elements in a that are greater than or equal to this element. The result is a list of the 'k' largest elements in 'a'.

The idea behing **k_largest_heap** is to maintain a min heap of the k largest elements seen so far. Whenever a new element is encountered, it is added to the heap only if it is greater than the minimum element in the heap. If the size of the heap exceeds k, then the minimum element is removed. The final result is the sorted k largest elements in descending order.

In [3]:
## Implementation test
a = get_random_array(1000, 10000)

assert sorted(k_largest_sort(a, 10)) == sorted(a)[-10:], "FAIL!"  
assert sorted(k_largest_quickselect(a, 10)) == sorted(a)[-10:], "FAIL!"  
assert sorted(k_largest_heap(a, 10)) == sorted(a)[-10:], "FAIL!"  

In [4]:
a = get_random_array(50000, 100)
K = 10

%timeit k_largest_sort(a, K)
%timeit k_largest_quickselect(a, K)
%timeit k_largest_heap(a, K)

6.47 ms ± 861 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.84 ms ± 885 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.3 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


---

### Exercise: compute distinct elements
You are given a list A of elements and you want to obtain the list of distict elements in A.

There are two possible algorithms to do this:

- Use ```list(set(A))```
- Sort A and then scan. Implement this as a function ```distinct(A)``` 

Compare these two approaches by varying the size of the array and the number of distinct elements.

In [5]:
def distinct(a):
    return sorted([x for i, x in enumerate(a) if x not in a[:i]])

In [6]:
## Implementation test
a = get_random_array(1000)

assert distinct(a) == sorted(list(set(a))), "FAIL!"

## What's the fastest approach?

In [7]:
a = get_random_array(10000, 10)

%timeit list(set(a))
%timeit distinct(a)

86.8 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
141 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

### Exercise: Pareto frontier of a set of points in 2-D space (aka Skyline problem)
We are given a set $S$ of $n$ 2D points.
A point $(x,y)$ dominates a point $(x',y')$ iff $𝑥'\leq 𝑥$ and $y'\leq 𝑦$. 
Our goal is to find the set $P$ of dominating points in $S$. 
This corresponds to find the Pareto frontier (or, equivalently, the skyline). 


This problem has a lot of [applications](https://en.wikipedia.org/wiki/Multi-objective_optimization) (and [here](https://en.wikipedia.org/wiki/Pareto_efficiency)).

The problem can be solved in $\Theta(n\log n)$ time.

To find $P$ we need to sort points in $S$ by $x$ in descending order, 
and if $x$′𝑠 the same by $y$ in descending order. This takes $\Theta(n\log n)$ time. 
Then, we do the following.

- Include first point in $P$ and remember this point as $𝑇$. 
- Iterates through the point (let $C$ current point):
* if $C$ is dominated by $T$, then skip $C$ and go to next point;
* Otherwise, include $C$ in $P$ and set $𝑇=𝐶$.

This step can be performed in linear time.

Implement the function ```pareto_frontier(S)```, which returns the pareto frontier $P$ of the points in $S$.


In [8]:
def pareto_frontier(S):
    S.sort(key=lambda point: (point[0], point[1]), reverse=True) # sort points in decreasing order
    P = [] # this will be the frontier (dominating points)
    T = S[0] 
    P.append(T)
    for C in S: # checks whether C dominates T, which means that C has a smaller or equal x-value and a smaller or equal y-value than T
        if (C[0] <= T[0]) and (C[1] <= T[1]):
            continue
        else: # if C dominates T, then T is updated to C and C is added to the Pareto frontier P
            P.append(C)
            T = C
    return list(reversed(P)) # the function returns the reversed list of P, because the points were added to P in reverse order

In [9]:
## Implementation test

S = [(6, 7.5), (7, 8), (8, 7), (2, 9), (3, 9.5), (1, 10), (4, 9), (5, 8)]

assert pareto_frontier(S) == [(1, 10), (3, 9.5), (4, 9), (7, 8), (8, 7)], "Fail!"